# Model A ALL questions - CoreNLP + TFIDF

## Prepare all data

In [1]:
f = open("data/dataset - fixed.csv", "r", encoding="utf-8")

first = True
questions_all = []
answers_all = []
grades_all = []
texts_all = []
for line in f:
    if first:
        first = False
        continue
    s = line.split(";")
    questions_all.append(s[10])
    answers_all.append(s[11])
    grades_all.append(s[14])
    texts_all.append(s[15])
    if s[15] == '1':
        print(line)
#test_answers = [answers_all[i] for i in range(len(questions_all)) if questions_all[i] == questions[0]]
#test_grades = [float(grades_all[i].replace(",", ".")) for i in range(len(questions_all)) if questions_all[i] == questions[0]]

## DATA[question_number] -> tuples of (queston, grade, answer, text)
old = questions_all[0]
DATA = []
data_tmp = []
for i in range(len(questions_all)):
    if questions_all[i] == old:
        triple = (questions_all[i], grades_all[i], answers_all[i], texts_all[i])
        data_tmp.append(triple)
    else:
        old = questions_all[i]
        DATA.append(data_tmp)
        data_tmp = []
        data_tmp.append((questions_all[i], grades_all[i], answers_all[i], texts_all[i]))
DATA.append(data_tmp)
        
print(len(DATA))
for i in DATA:
    print(len(i))

12
81
81
70
76
67
69
71
69
62
65
68
71


## Split train and test data

In [2]:
ratio = 0.8 #0.8 train data, 0.2 test data FOR EACH QUESTION

DATA_train = []
DATA_test = []

for i in DATA:
    split = int(len(i) * ratio)
    DATA_train.append(i[:split])
    DATA_test.append(i[split:])

#Test and verification
#for i in range(len(DATA)):
#    print(len(DATA[i]))
#    print(len(DATA_train[i]) + len(DATA_test[i]))

## TF-IDF

## Preparation

In [3]:
import requests
url = 'http://localhost:9000/?properties={"annotators": "openie,coref", "outputFormat": "json", "openie.resolve_coref": "true"}'
data = "Sarah jumped over the table. She got hurt."
response = requests.post(url, data=data)

In [4]:
for s in response.json()["sentences"]:
    for i in s["openie"]:
        print("Subject:", i["subject"], " | Relation:", i["relation"], " | Object:", i["object"])

Subject: Sarah  | Relation: jumped over  | Object: table
Subject: Sarah  | Relation: got  | Object: hurt


In [5]:
def openie_extract(text, resolve_coref=True):
    import requests
    if resolve_coref:
        url = 'http://localhost:9000/?properties={"annotators": "tokenize,ssplit,pos,lemma,openie,coref", "outputFormat": "json", "openie.resolve_coref": "true", "openie.triple.strict": "true"}'
    else:
        url = 'http://localhost:9000/?properties={"annotators": "tokenize,ssplit,pos,lemma,openie,coref", "outputFormat": "json", "openie.resolve_coref": "false", "openie.triple.strict": "true"}'
    data = text
    response = requests.post(url, data=data)
    response.encoding = "utf-8"
    triples = []
    for s in response.json()["sentences"]:
        for i in s["openie"]:
            #print("Subject:", i["subject"], " | Relation:", i["relation"], " | Object:", i["object"])
            #triples.append((lemmatize([i["subject"]])[0], lemmatize([i["relation"]])[0], lemmatize([i["object"]])[0]))
            triples.append((i["subject"], i["relation"], i["object"]))
    return triples

In [6]:
## DATA[question_number] -> tuples of (queston, grade, answer, text)

BASE_TRIPLES = []
for i in DATA_train:
    data = i[0][3] + " "#answers[i] + " " + texts[i]
    for j in i: #loop through all tuples
        data += j[2] + " "
    BASE_TRIPLES.append(openie_extract(data.encode("utf8")))

In [7]:
import string
import nltk
from nltk.stem import WordNetLemmatizer
def getTokens(text):
    lowered = text.lower()
    table = text.maketrans({key: None for key in string.punctuation})
    lowered = lowered.translate(table)
    return nltk.word_tokenize(lowered)

def lemmatize(tokens):
    lemmatizer = WordNetLemmatizer()
    lemmas = [lemmatizer.lemmatize(token) for token in tokens]
    return lemmas

def preprocess(text):
    tokens = getTokens(text)
    lemmatizer = WordNetLemmatizer()
    lemmas = [lemmatizer.lemmatize(token) for token in tokens]
    return " ".join(lemmas)
    #pos_tag = nltk.pos_tag(lemmas)
    #print(pos_tag)
    #return " ".join([pt[0] for pt in pos_tag if pt[1] == "NN" or pt[1][0:2] == "VB" or pt[1] == "JJ"])


#pre_answers = [preprocess(ans) for ans in answers]
#pre_texts = [preprocess(tex) for tex in texts]

#pre_answers[question_number] -> array of preprocessed answers
#pre_text[question_number] -> preprocessed text for that question

## DATA[question_number] -> tuples of (queston, grade, answer, text)

pre_answers = []
pre_texts = []
for i in DATA_train:
    pre_answers.append([preprocess(ans[2]) for ans in i])
    pre_texts.append(preprocess(i[0][3]))

DATA -> ALL data from csv, DATA[question_number] -> triples of (question, grade, answer, text)

answers -> answers for model A (one for each question)

texts -> texts for model A (one for each question)

## Predicting

In [8]:
#weights = vect.transform(test_answers)

from sklearn.feature_extraction.text import TfidfVectorizer

correct = 0
TP = 0
TN = 0
FP = 0
FN = 0

all_count = 0

#for F1 scoring
true_grades = []
predicted_grades = []

## DATA[question_number] -> triples of (queston, grade, answer, text)
for d in range(len(DATA_test)):
    test_answers = [a[2] for a in DATA_test[d]]
    test_grades = [float(a[1]) for a in DATA_test[d]]
    
    true_grades += test_grades

    vect = TfidfVectorizer() # parameters for tokenization, stopwords can be passed
    #tfidf = vect.fit_transform([pre_answers[d], pre_texts[d]])
    tfidf = vect.fit_transform([pre_texts[d]] + pre_answers[d])
    
    weights = vect.transform([preprocess(ta) for ta in test_answers])
    predict = tfidf * weights.T

    #Shiranna feels excited and scared as the shuttle is taking off and it even affects her heart-rate and her temperature.
    #test_answers = ["Excited, scared. It affected her heart rate and temp"]

    
    for i in range(len(test_answers)):
        all_count += 1 # only for statistics at the end
        
        test_answer = test_answers[i]
        triples = openie_extract(test_answer.encode("utf8"))

        p = 0
        p_tfidf = 0
        p_triples = 0

        #if len(triples) < 1:
        #prediction = max(predict[0,i], predict[1,i])
        prediction = max(predict[:,i])
        if prediction > 0.35: #0.5
            p_tfidf = 1
        elif prediction > 0.2: #0.3
            p_tfidf = 0.5
        else:
            p_tfidf = 0
        #else:
        for bt in BASE_TRIPLES[d]:
            for t in triples:
                if t[0] == bt[0] or t[1] == bt[1] or t[2] == bt[2]:
                #if (t[0] == bt[0] and t[1] == bt[1]) or (t[0] == bt[0] and t[2] == bt[2]) or (t[1] == bt[1] and t[2] == bt[2]):
                    p_triples += 0.5
        p_triples = p_triples*4 / max(len(triples), 1)
        prediction = 0
        if p_triples >= 1:
            p_triples = 1
        elif p_triples >= 0.5:
            p_triples = 0.5
        else:
            p_triples = 0

        p = (p_triples + p_tfidf) / 2
        if p > 0 and p < 1 and p != 0.5:
            p = 0.5
            
        #p = p_tfidf
        
        predicted_grades.append(p)
        #print("Predicted:", p, " Real:", test_grades[i], " --- ", len(triples))
        
        if p == test_grades[i]:
            correct += 1
            if p == 1 or p == 0.5:
                TP += 1
            else:
                TN += 1
        else:
            #print(len(triples))
            if p == 1 or p == 0.5:
                FP += 1
            else:
                FN += 1
    
print("Correct: ", correct, "/", all_count)

Correct:  114 / 176


[texts[0] + answers[0]] 0.3 0.2 -> 38/81 correct

[texts[0], answers[0]]  0.3 0.2 -> 42/81 correct 

CoreNLP openie+coref /w TFIDF -> 32/81 correct

CoreNLP openie /w TFIDF ->  40/81 correct

CoreNLP openie relative scoring /w TFIDF -> 36/81 correct

CoreNLP openie+coref relative scoring /w TFIDF -> 33/81 correct

CoreNLP (not strict) openie+coref relative scoring AND TFIDF | AVG -> 43/81 correct

CoreNLP openie+coref relative scoring AND TFIDF | AVG -> 40/81 correct

CoreNLP openie+coref(with and without) relative scoring AND TFIDF | AVG -> 41/81 correct

## F1 Scoring

In [9]:
from sklearn.metrics import f1_score
tg = [i*2 for i in true_grades]
pg = [i*2 for i in predicted_grades]
#print(tg)
#print(pg)
print("Vsa vprašanja (mikro, makro): ", f1_score(tg,pg,average="micro"), f1_score(tg,pg,average="macro"))

Vsa vprašanja (mikro, makro):  0.6477272727272727 0.41571002719682665


### F1 score: openIE + coref relative scoring averaged with TFIDF
Correct:  114 / 176

Vsa vprašanja (mikro, makro):  0.6477272727272727 0.41571002719682665

### F1 score: openIE + coref relative scoring WITHOUT TFIDF
Correct:  116 / 176

Vsa vprašanja (mikro, makro):  0.6590909090909091 0.35599276345545

### F1 score: only TFIDF
Correct:  129 / 176

Vsa vprašanja (mikro, makro):  0.7329545454545454 0.4020678828560602